# RAG on Medical PDFs - Nutrition Restrictions Extraction

This notebook extracts nutrition restrictions and dietary guidelines from medical PDF documents.
The output is a structured JSON that can be used in other RAG systems for personalized nutrition advice.

In [1]:
# LangChain imports for RAG
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Hybrid search imports
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# Ollama LLM
from langchain_ollama import OllamaLLM

# Additional utilities
import os
import json
from pathlib import Path
from typing import List, Dict, Any
from pydantic import BaseModel, Field

# Path to PDF documents
PDF_FOLDER = "../data/pdfs/"

## 1. Load PDF Documents

We load all PDF files from the `data/pdfs/` folder. These contain medical nutrition guidelines that will be used to extract dietary restrictions and recommendations.

In [2]:
# List all PDF files in the folder
pdf_files = list(Path(PDF_FOLDER).glob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files:")
for f in pdf_files:
    print(f"  - {f.name}")

# Load all PDFs
all_documents = []
for pdf_path in pdf_files:
    try:
        loader = PyPDFLoader(str(pdf_path))
        docs = loader.load()
        # Add source filename to metadata
        for doc in docs:
            doc.metadata["source_file"] = pdf_path.name
        all_documents.extend(docs)
        print(f"Loaded {len(docs)} pages from {pdf_path.name}")
    except Exception as e:
        print(f"Error loading {pdf_path.name}: {e}")

print(f"\nTotal pages loaded: {len(all_documents)}")

Found 5 PDF files:
  - 10-61474-ncs-2025-00004.pdf
  - Eating-Well-with-Parkinsons-Disease.pdf
  - northwestern-medicine-summary-of-nutritional-guide-for-parkinsons-disease.pdf
  - Nutrition-Parkinsons_A4Manual_Aug2021.pdf
  - PARKINSON1614-Guideline-Nutrition-A4-ENG.pdf
Loaded 13 pages from 10-61474-ncs-2025-00004.pdf
Loaded 36 pages from Eating-Well-with-Parkinsons-Disease.pdf
Loaded 2 pages from northwestern-medicine-summary-of-nutritional-guide-for-parkinsons-disease.pdf
Loaded 44 pages from Nutrition-Parkinsons_A4Manual_Aug2021.pdf
Loaded 39 pages from PARKINSON1614-Guideline-Nutrition-A4-ENG.pdf

Total pages loaded: 134


## 2. Chunk Documents for RAG

Unlike tabular nutrition data, PDF documents contain long-form text that needs to be split into smaller chunks for effective retrieval. We use:
- **Chunk size**: 1000 characters (captures meaningful paragraphs)
- **Overlap**: 200 characters (maintains context across chunks)

In [3]:
# Create text splitter for chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Split documents into chunks
chunks = text_splitter.split_documents(all_documents)

print(f"Created {len(chunks)} chunks from {len(all_documents)} pages")
print(f"Average chunk length: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} characters")

# Preview a chunk
print("\n--- Example chunk ---")
print(chunks[5].page_content[:500] + "...")
print(f"\nMetadata: {chunks[5].metadata}")

Created 356 chunks from 134 pages
Average chunk length: 797 characters

--- Example chunk ---
possible causes including pesticides and brain injury. In familial cases of PD, however, a set of genes have been identified 
as causes of PD, including the α- synuclein gene, Parkin, PTEN-induced kinase 1, and Leucine-rich repeat kinase 2 . To 
date, there is no cure for this disease, except for some palliative treatments such as dopa/dopamine therapy. Dopamine 
administration from external sources is effective only for a couple of years; after that, dyskinesia and other neurological 
complicat...

Metadata: {'source': '..\\data\\pdfs\\10-61474-ncs-2025-00004.pdf', 'page': 0, 'source_file': '10-61474-ncs-2025-00004.pdf'}


## 3. Create Embeddings and Vector Store

We use `BAAI/bge-small-en-v1.5` for embeddings - optimized for retrieval tasks with good performance on medical/scientific text.

In [4]:
# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Embedding model loaded: BAAI/bge-small-en-v1.5")
print(f"Embedding dimension: {len(embeddings.embed_query('test'))}")

d:\Programming\ai_ds_bootcamp\nutrition-ai-assistent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding model loaded: BAAI/bge-small-en-v1.5
Embedding dimension: 384


## 4. Create Vector Store

In [5]:
# Create FAISS vector store from chunks
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings,
    distance_strategy=DistanceStrategy.COSINE
)

print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 356 vectors


In [6]:
# Optional: Save vector store for later use
VECTORSTORE_PATH = "../data/processed/medical_pdfs_vectorstore"

# Uncomment to save
vectorstore.save_local(VECTORSTORE_PATH)
print(f"Vector store saved to {VECTORSTORE_PATH}")

Vector store saved to ../data/processed/medical_pdfs_vectorstore


## 5. Create Hybrid Retriever

Combining BM25 (keyword) and vector search for better retrieval of medical terminology.

In [7]:
# Create BM25 retriever for keyword matching
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5

# Create vector retriever with MMR for diverse results
vector_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k": 20,
        "lambda_mult": 0.7
    }
)

# Combine both retrievers
retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.4, 0.6]
)

print("Hybrid retriever created (BM25 + Vector with MMR)")

# Test retrieval
test_query = "What foods should be avoided with Parkinson's disease?"
retrieved_docs = retriever.invoke(test_query)
print(f"\nTest query: {test_query}")
print(f"Retrieved {len(retrieved_docs)} documents")

Hybrid retriever created (BM25 + Vector with MMR)

Test query: What foods should be avoided with Parkinson's disease?
Retrieved 9 documents


## 6. Define Simple JSON Output Schema for Nutrition Parameters

The output is a simple set of nutrition constraints that can be passed directly to the recipes/nutrition RAG for filtering recommendations.

In [ ]:
# Simple JSON schema for nutrition parameters
# Each parameter has: target value, min/max limits, unit, and priority

NUTRITION_PARAMS_SCHEMA = {
    "condition": "string - medical condition",
    "daily_targets": {
        "calories": {"min": 0, "max": 0, "unit": "kcal"},
        "protein_g": {"min": 0, "max": 0, "unit": "g", "notes": ""},
        "fiber_g": {"min": 0, "max": 0, "unit": "g"},
        "sodium_mg": {"min": 0, "max": 0, "unit": "mg"},
        "water_ml": {"min": 0, "max": 0, "unit": "ml"}
    },
    "per_meal_limits": {
        "protein_g": {"max": 0, "notes": "timing with medication"}
    },
    "increase": ["list of nutrients/foods to prioritize"],
    "limit": ["list of nutrients/foods to reduce"],
    "avoid": ["list of foods to exclude"],
    "timing_notes": ["medication-related timing advice"]
}

print("Simple Nutrition Parameters Schema:")
print(json.dumps(NUTRITION_PARAMS_SCHEMA, indent=2))

## 7. Connect to Ollama LLM

Prerequisites:
1. Install Ollama: https://ollama.ai/
2. Pull a model: `ollama pull llama3.2`
3. Start Ollama: `ollama serve`

In [10]:
# Initialize Ollama LLM
llm = OllamaLLM(
    model="llama3.2",
    temperature=0.3,  # Lower temperature for more consistent JSON output
)

# Test connection
test_response = llm.invoke("Say 'Connection successful!' if you can read this.")
print(test_response)

Connection successful!


## 8. Create RAG Chain for Nutrition Restrictions Extraction

The prompt is designed to extract structured nutrition restrictions from medical documents and return valid JSON.

In [ ]:
# System prompt for extracting simple nutrition parameters
extraction_system_prompt = """You are a medical nutrition specialist. Extract nutrition parameters from medical documents.

OUTPUT FORMAT - Return ONLY valid JSON with this exact structure:
{{
    "condition": "Parkinson's Disease",
    "daily_targets": {{
        "calories_kcal": {{"min": number, "max": number}},
        "protein_g": {{"min": number, "max": number}},
        "fiber_g": {{"min": number, "max": number}},
        "sodium_mg": {{"min": number, "max": number}},
        "water_ml": {{"min": number, "max": number}},
        "calcium_mg": {{"min": number, "max": number}},
        "vitamin_d_iu": {{"min": number, "max": number}}
    }},
    "per_meal_limits": {{
        "protein_g": {{"max": number, "reason": "string"}}
    }},
    "increase": ["nutrient or food to eat more of"],
    "limit": ["nutrient or food to reduce"],
    "avoid": ["foods to completely avoid"],
    "timing_notes": ["when to eat relative to medication"]
}}

RULES:
1. Use numbers only (no units in values), use 0 if not specified
2. Extract from context only - don't invent values
3. For Parkinson's: pay attention to protein timing with Levodopa
4. Include fiber (for constipation) and hydration recommendations
5. Return ONLY the JSON, no other text

CONTEXT:
{context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", extraction_system_prompt),
    ("human", "{input}")
])

# Create the RAG chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("RAG chain created for nutrition parameters extraction!")

In [12]:
def extract_nutrition_restrictions(query: str) -> Dict[str, Any]:
    """
    Extract nutrition restrictions from medical PDFs based on a query.
    Returns a structured JSON dictionary.
    """
    response = rag_chain.invoke({"input": query})
    raw_answer = response["answer"]
    
    # Try to parse as JSON
    try:
        # Clean up the response - remove any markdown code blocks
        cleaned = raw_answer.strip()
        if cleaned.startswith("```json"):
            cleaned = cleaned[7:]
        if cleaned.startswith("```"):
            cleaned = cleaned[3:]
        if cleaned.endswith("```"):
            cleaned = cleaned[:-3]
        cleaned = cleaned.strip()
        
        result = json.loads(cleaned)
        return {
            "success": True,
            "data": result,
            "sources": [doc.metadata.get("source_file", "unknown") for doc in response["context"]]
        }
    except json.JSONDecodeError as e:
        return {
            "success": False,
            "raw_response": raw_answer,
            "error": str(e),
            "sources": [doc.metadata.get("source_file", "unknown") for doc in response["context"]]
        }


def save_restrictions_to_json(restrictions: Dict[str, Any], output_path: str):
    """Save extracted restrictions to a JSON file."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(restrictions, f, indent=2, ensure_ascii=False)
    print(f"Restrictions saved to: {output_path}")


print("Extraction functions defined!")

Extraction functions defined!


## 9. Extract Nutrition Restrictions

Now we extract all nutrition restrictions from the medical PDFs.

In [ ]:
# Extract nutrition parameters for Parkinson's disease
query = """What are the recommended daily nutrition parameters for Parkinson's disease patients?
Extract: calories, protein (daily and per-meal limits), fiber, sodium, water intake, calcium, vitamin D.
Also list foods to increase, limit, and avoid. Include any medication timing notes for Levodopa."""

print("Extracting nutrition parameters...")
result = extract_nutrition_restrictions(query)

if result["success"]:
    print("✓ Successfully extracted parameters!\n")
    print(json.dumps(result["data"], indent=2))
else:
    print("✗ Could not parse as JSON")
    print(f"Error: {result['error']}\n")
    print("Raw response:")
    print(result["raw_response"])

In [ ]:
# Save nutrition parameters to JSON file
OUTPUT_PATH = "../data/processed/nutrition_params_parkinsons.json"

if result["success"]:
    save_restrictions_to_json(result["data"], OUTPUT_PATH)
    print("\nThis JSON can be loaded by the nutrition RAG to filter recipes!")
else:
    fallback_data = {
        "condition": "Parkinson's Disease",
        "raw_text": result["raw_response"],
        "note": "Manual parsing needed"
    }
    save_restrictions_to_json(fallback_data, OUTPUT_PATH.replace(".json", "_raw.json"))

## 10. Example: View Extracted Parameters

In [ ]:
# Display the extracted parameters in a readable format
if result["success"]:
    data = result["data"]
    
    print("=" * 50)
    print(f"CONDITION: {data.get('condition', 'Unknown')}")
    print("=" * 50)
    
    print("\n📊 DAILY TARGETS:")
    for nutrient, values in data.get("daily_targets", {}).items():
        if isinstance(values, dict):
            print(f"  {nutrient}: {values.get('min', 0)} - {values.get('max', 0)}")
    
    print("\n🍽️ PER-MEAL LIMITS:")
    for nutrient, values in data.get("per_meal_limits", {}).items():
        if isinstance(values, dict):
            print(f"  {nutrient}: max {values.get('max', 0)} ({values.get('reason', '')})")
    
    print("\n✅ INCREASE (eat more):")
    for item in data.get("increase", []):
        print(f"  • {item}")
    
    print("\n⚠️ LIMIT (reduce):")
    for item in data.get("limit", []):
        print(f"  • {item}")
    
    print("\n❌ AVOID:")
    for item in data.get("avoid", []):
        print(f"  • {item}")
    
    print("\n⏰ TIMING NOTES:")
    for note in data.get("timing_notes", []):
        print(f"  • {note}")

In [ ]:
# The output JSON structure - ready for use in nutrition RAG
print("Output JSON file:", OUTPUT_PATH)
print("\nThis file contains nutrition parameters that can be used to:")
print("1. Filter recipes by protein content per meal")
print("2. Prioritize high-fiber foods")
print("3. Exclude foods from the 'avoid' list")
print("4. Apply timing rules relative to medication")

## 11. Using Nutrition Parameters in Recipe RAG

The JSON output can filter recipes based on the extracted constraints.

In [ ]:
# Example: How to use nutrition params in recipe RAG

def load_nutrition_params(json_path: str) -> Dict[str, Any]:
    """Load nutrition parameters from JSON."""
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def check_recipe_fits_params(recipe_nutrition: Dict, params: Dict) -> bool:
    """Check if a recipe fits within the nutrition parameters."""
    daily = params.get("daily_targets", {})
    
    # Check protein limit per meal
    meal_limits = params.get("per_meal_limits", {})
    if "protein_g" in meal_limits:
        max_protein = meal_limits["protein_g"].get("max", 999)
        if recipe_nutrition.get("protein_g", 0) > max_protein:
            return False
    
    # Check sodium
    if "sodium_mg" in daily:
        max_sodium = daily["sodium_mg"].get("max", 999999)
        if recipe_nutrition.get("sodium_mg", 0) > max_sodium / 3:  # per meal
            return False
    
    return True

# Example usage:
# params = load_nutrition_params("../data/processed/nutrition_params_parkinsons.json")
# print("Foods to increase:", params.get("increase", []))
# print("Foods to avoid:", params.get("avoid", []))
# print("Per-meal protein limit:", params.get("per_meal_limits", {}).get("protein_g", {}))

print("Helper functions ready for recipe filtering!")